### How is the Graph Structure of a PGM learned?

In an interview for *The World from another Point of View*, Richard Feynman says:

"
I read Dirac's book, and he had these problems that nobody knew how to solve. I couldn't understand the book very well because I really wasn't up to it. But there, in the last paragraph at the end of the book, it said 'some new ideas are here needed.' And so there I was - some new ideas are needed? OK, so I started to think of new ideas.
"

I'm no Dirac, but at this point in covering PGMs, I'd like to make the same statement: *Some new ideas are here needed.* Learning the graph structure is *hard*, and you, a possible Feynman, could provide those ideas. 

